In [ ]:
import jax_sbgeom as jsb
%load_ext autoreload
%autoreload 2
import h5py
import jax.numpy as jnp
import jax
jax.config.update("jax_enable_x64", True)
import numpy as onp
import sys 
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)
vmec_file = "/home/tbogaarts/stellarator_paper/base_data/vmecs/helias5_vmec.nc4"
import StellBlanket.SBGeom as SBGeom
from jax_sbgeom.flux_surfaces.flux_surfaces_base import _cartesian_position_interpolated_jit, _cylindrical_position_interpolated, _cartesian_position_interpolated_grad, ToroidalExtent
from tests.flux_surfaces.flux_surface_base import test_position, _get_flux_surfaces, _sampling_grid, _1d_sampling_grid, test_normals, test_meshing_surface, test_principal_curvatures


In [ ]:
test_meshing_surface(vmec_file)

NameError: name 'test_meshing_surface' is not defined

In [5]:
def _get_flux_surfaces(vmec_file):
    fs_jax    = jsb.flux_surfaces.FluxSurface.from_hdf5(vmec_file)
    fs_sbgeom = SBGeom.VMEC.Flux_Surfaces_From_HDF5(vmec_file)
    return fs_jax, fs_sbgeom

fs_jax, fs_sbgeom = _get_flux_surfaces(vmec_file)  # just to compile

In [18]:
from jax_sbgeom.flux_surfaces import mesh_surface
from jax_sbgeom.flux_surfaces.flux_surface_meshing import _mesh_surfaces_closed

nt = 50 
nph =60
positions_c, connectivity_c = mesh_surface(fs_jax, 1.0, ToroidalExtent(0.0, jnp.pi), n_theta = nt, n_phi = nph, normals_facing_outwards=True)

In [20]:
positions_t_p = onp.array(positions_c.reshape(nt, nph, 3))

In [35]:
def _flip_vertices_theta(positions, n_theta, n_phi):
    positions_rs = positions.reshape(n_theta, n_phi, 3)

    first = jnp.take(positions_rs, indices = 0, axis = 0)
    rest = jnp.flip(jnp.take(positions_rs, indices = jnp.arange(1, n_theta), axis = 0), axis = 0)
    positions_rs_flipped = jnp.concatenate([first[None, :, :], rest], axis = 0)
    return positions_rs_flipped.reshape(-1, 3)



In [36]:
positions_2 = _flip_vertices_theta(positions_c, nt, nph)

In [41]:
import numpy as np 
import pyvista as pv
plotter = pv.Plotter()

plotter.add_mesh(pv.PolyData(np.array(positions_c)))
plotter.add_mesh(pv.PolyData(np.array(positions_2).reshape(-1,3)), color = 'red', opacity = 0.4)

plotter.show()

Widget(value='<iframe src="http://localhost:44185/index.html?ui=P_0x7b0f0c7cc8c0_9&reconnect=auto" class="pyvi…

In [8]:
import numpy as np
mesh_sbgeom = fs_sbgeom.Mesh_Surface(0.5, 0.0, 100, 100, 0.0, np.pi)

In [9]:
import pyvista as pv
import numpy as np
plotter = pv.Plotter()
pts = np.asarray(positions_c)
conn = np.asarray(connectivity_c, dtype=np.int64)

# PyVista expects a flat "faces" array: [n_vertices, v0, v1, v2, n_vertices, v0, v1, v2, ...]
faces = np.hstack([np.full((conn.shape[0], 1), 3, dtype=np.int64), conn]).ravel()

mesh = pv.PolyData(pts, faces)

mesh.compute_normals(cell_normals=True, inplace=True)
plotter.add_mesh(mesh, color='white', show_edges=True)
plotter.add_mesh(mesh_sbgeom.to_pyvista())
plotter.add_arrows(mesh.cell_centers().points, mesh['Normals'], mag=0.1, color='red')
plotter.show()

Widget(value='<iframe src="http://localhost:44185/index.html?ui=P_0x7b0fcbf30560_0&reconnect=auto" class="pyvi…

In [ ]:
pos_list = [positions[i,...] for i in range(positions.shape[0])]
conn_list = [connectivity[i,...] for i in range(connectivity.shape[0])]

NameError: name 'positions' is not defined

In [ ]:
def reverse_theta_single(m_vec, n_vec, coeff_vec, cosine_sign : bool):
    assert coeff_vec.ndim == 1

    assert m_vec.shape == n_vec.shape == coeff_vec.shape
    # Create a "key" for each coefficient: (m, n)
    keys = jnp.stack([m_vec, n_vec], axis=1)
    
    # Create a lookup table: key -> index
    # We'll use broadcasting to match (m, -n)
    reversed_keys = jnp.stack([m_vec, -n_vec], axis=1)  # target keys
    
    reversed_keys_mod = jnp.where(keys[:,0:1] > 0, reversed_keys, keys)
    
    
    matches = jnp.all(keys[:, None, :] == reversed_keys_mod[None, :, :], axis=-1)

    assert jnp.all(jnp.any(matches, axis=0))
    
    # For each column in reversed_keys, find row index in keys
    idx_map = jnp.argmax(matches, axis=0)
    
    
    # Build new Z_vec: -Z_vec[mapped_index]
    new_coeff_vec = jnp.where(cosine_sign, coeff_vec[idx_map], -coeff_vec[idx_map])
    
    return new_coeff_vec

reverse_theta_total  = jax.vmap(reverse_theta_single, in_axes=(None, None, 0, None), out_axes=0)

print(reverse_theta_single(fs_jax.data.mpol_vector, fs_jax.data.ntor_vector, fs_jax.data.Zmns[-1,:], False))

[ 0.00000000e+00  8.87110545e-01  1.23708737e-02  1.72806543e-02
  7.60098238e-03  1.32181648e-03  1.95386064e-04 -7.44682825e-04
  1.36691783e-03  8.74813832e-04 -5.08738278e-04 -3.92411355e-04
 -6.53033382e-05  7.09699578e-05  8.20513816e-04  1.19774864e-03
 -6.53292147e-04 -1.81101315e-03  8.28432571e-04  6.83905103e-03
 -1.07966468e-02 -2.28158277e-03  2.50990585e-02 -3.60128844e-02
  7.72870451e-02  2.88472946e+00  1.27724435e+00  3.46642109e-01
  6.62636923e-03 -6.81423883e-03  6.84131125e-03 -1.05130135e-03
 -2.70430745e-03 -3.50347409e-04  1.42332070e-03 -2.29197780e-04
 -1.46542145e-03 -6.82593939e-04 -5.56001032e-05  2.76918443e-05
  1.62241638e-04  3.27721042e-04  5.11637618e-04 -1.58881597e-04
 -8.93269609e-04  9.12980413e-04  5.86440482e-04  3.14205820e-03
  1.05481850e-02  5.33875177e-03  1.27347838e-01  7.92634724e-02
 -1.86775711e-01 -7.50946720e-02 -1.70578701e-02  4.82778171e-03
  7.32824635e-04 -1.78270341e-03 -2.91315391e-04  2.91381411e-04
  2.85851678e-04 -2.27273

In [ ]:
fs_jax.data.ntor_vector

Array([  0,   3,   6,   9,  12,  15,  18,  21,  24,  27,  30,  33,  36,
       -36, -33, -30, -27, -24, -21, -18, -15, -12,  -9,  -6,  -3,   0,
         3,   6,   9,  12,  15,  18,  21,  24,  27,  30,  33,  36, -36,
       -33, -30, -27, -24, -21, -18, -15, -12,  -9,  -6,  -3,   0,   3,
         6,   9,  12,  15,  18,  21,  24,  27,  30,  33,  36, -36, -33,
       -30, -27, -24, -21, -18, -15, -12,  -9,  -6,  -3,   0,   3,   6,
         9,  12,  15,  18,  21,  24,  27,  30,  33,  36, -36, -33, -30,
       -27, -24, -21, -18, -15, -12,  -9,  -6,  -3,   0,   3,   6,   9,
        12,  15,  18,  21,  24,  27,  30,  33,  36, -36, -33, -30, -27,
       -24, -21, -18, -15, -12,  -9,  -6,  -3,   0,   3,   6,   9,  12,
        15,  18,  21,  24,  27,  30,  33,  36, -36, -33, -30, -27, -24,
       -21, -18, -15, -12,  -9,  -6,  -3,   0,   3,   6,   9,  12,  15,
        18,  21,  24,  27,  30,  33,  36, -36, -33, -30, -27, -24, -21,
       -18, -15, -12,  -9,  -6,  -3,   0,   3,   6,   9,  12,  1

In [ ]:
jnp.where(fs_jax.data.ntor_vector > 0,  )

(Array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  26,
         27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  51,  52,
         53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  76,  77,  78,
         79,  80,  81,  82,  83,  84,  85,  86,  87, 101, 102, 103, 104,
        105, 106, 107, 108, 109, 110, 111, 112, 126, 127, 128, 129, 130,
        131, 132, 133, 134, 135, 136, 137, 151, 152, 153, 154, 155, 156,
        157, 158, 159, 160, 161, 162, 176, 177, 178, 179, 180, 181, 182,
        183, 184, 185, 186, 187, 201, 202, 203, 204, 205, 206, 207, 208,
        209, 210, 211, 212, 226, 227, 228, 229, 230, 231, 232, 233, 234,
        235, 236, 237, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260,
        261, 262, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286,
        287], dtype=int64),)

In [ ]:
pos_c, c_c = _concatenate_meshes(pos_list, conn_list)

NameError: name '_concatenate_meshes' is not defined

In [ ]:
import meshio 

meshio.Mesh(pos_c, [("triangle", c_c)]).write("ff.stl")

In [ ]:
fs_sbgeom.Mesh_Surfaces_Closed( 0.5, 1.0, 0.0, 0.0, 50,60, 0.0, np.pi).to_meshio().write("ff_sbgeom.stl")

In [ ]:
fs_sbgeom.Mesh_Surface(1.0, 0.0, 50, 60, 0.0, np.pi, True).to_meshio().write('ff_sbgeom2.stl')